In [1]:
import pickle
f=  open ('./char_embedding.pkl', 'rb')
dict_char = pickle.load(f)
f.close()

In [2]:
dict_char_idx = dict()
dict_idx_char = dict()
for i, char in enumerate(dict_char.keys()):
    dict_char_idx[char] = i
for i, char in enumerate(dict_char.keys()):
    dict_idx_char[i] = char

In [3]:
import torch
from torch import nn
from torch.autograd import Variable as V
import torch.optim as optim
from torch.nn.functional import log_softmax 
import random
import torch.nn.functional as F
import numpy as np
from torch.nn.functional import softmax 

In [4]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.gru = nn.GRU(self.input_size, self.hidden_size,dropout = 0.5)
        self.h2o = nn.Linear(self.hidden_size, self.output_size)
        self.softmax = nn.LogSoftmax() 
    def forward(self, input):
        out,_ = self.gru(input)
        out = self.h2o(out).squeeze(0).squeeze(0)
        output = self.softmax(out)
        return output
    def initHidden(self):
        return torch.zeros(1, self.hidden_size)

In [5]:
def predict_name_ran(char1) :
    char2=''
    char3=''
    char4='' 
    char2 = predict_char_ran10(char1)
    while char2 == '。':
        char2 = predict_char_ran10(char1)
    if char2 != '。' :
        char3 = predict_char_ran10(char2)
    if char3 != '。' :
        char4 = predict_char(char3)

    name = char1 + char2 + char3 + char4    
    return name

In [6]:
def predict_char(char):
    test = dict_char[char]
    test_tensor = torch.from_numpy(test).unsqueeze(0).unsqueeze(0).cuda()
    #print (test_tensor.shape)
    gen_test = model(test_tensor)
 
    k = int(torch.max(gen_test,0)[1])
    
    return dict_idx_char[k]

In [7]:
def predict_char_ran10(char): 
    test = dict_char[char] 
    test_tensor = torch.from_numpy(test).unsqueeze(0).unsqueeze(0).cuda() 
    gen_test = model(test_tensor) 
    x = torch.topk(gen_test,10)[1] 
    p = torch.topk(gen_test,10)[0] 
    p = softmax(p.float())
    
    x2 = []
    for i in list(x):
        x2.append(int(i)) 

    sum = 0
    p2 = []
    for i in list(p):
        p2.append(float(i))
        sum += float(i) 
    
    ac = 1-sum
    p2[0] += ac
    
    k = np.random.choice(range(10) , p=p2)
    index = x2[k]
    return dict_idx_char[index]

In [8]:
INPUT_DIM = 256
HIDDEN_DIM = 64
TARGET_DIM = len(dict_char.keys())
teacher_force = 0.87
forget_end = 0.1

model = RNN(INPUT_DIM, HIDDEN_DIM, TARGET_DIM).cuda()
model = torch.load('model-none-max-31.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\torch\nn\modules\rnn.py:50: UserWarning: dropout option adds dropout after all but last recurrent layer, so non-zero dropout expects num_layers greater than 1, but got dropout=0.5 and num_layers=1
  "num_layers={}".format(dropout, num_layers))
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.rnn.GRU' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)
C:\ProgramData\Anaconda3\lib\site-packages\torch\serialization.py:657: SourceChangeWarning: source code of class 'torch.nn.modules.linear.Linear' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warni

In [9]:
predict_name_ran('林')

AttributeError: 'GRU' object has no attribute '_flat_weights'